# Attempt to fine-tune GPT3 to translate between languages
The first language will be our made up language (see notes on get_bible as to why we think GPT is cheating)
The thesis is that fine-tuning will cause GPT3 to learn the fake language as a new language by assigning the embeddings of the
new words essentially to the same dimensional space as the english.  Should be trivial to translate it back as it is a word 
for word translation.



In [ ]:
# Q. what is John 3:2?


In [ ]:
from lib.config import get_config
import pandas as pd

bible = pd.read_csv('data/bible.bbe.csv')

# Assuming the gospels have a lot of overlap include the synoptic gospels plus John to train
# then predict Mark (which should be easy as Matthew and Luke may have copied from him)
train = bible[bible['book'].isin(['MAT','LUK','JHN'])]
validate = bible[bible['book'].isin(['MRK'])]

In [223]:
train[train['book'] == 'JHN']

,0,book,chapter,verse,eng-web,eng-asv,eng-kjv2006,engBBE,engWycliffe,birrig
26113,JHN 1:1,JHN,1,1,"In the beginning was the Word, and the Word wa...","In the beginning was the Word, and the Word wa...","In the beginning was the Word, and the Word wa...","From the first he was the Word, and the Word w...","In the bigynnyng was the word, and the word wa...","Svun lxi sovzl xi hez lxi Huvw, erw lxi Huvw h..."
26114,JHN 1:2,JHN,1,2,The same was in the beginning with God.,The same was in the beginning with God.,The same was in the beginning with God.,This Word was from the first in relation with ...,This was in the bigynnyng at God.,Lxoz Huvw hez svun lxi sovzl or vipelour holx ...
26115,JHN 1:3,JHN,1,3,"All things were made through him. Without him,...",All things were made through him; and without ...,All things were made by him; and without him w...,"All things came into existence through him, an...","Alle thingis weren maad bi hym, and withouten ...","Epp lxorgz keni orlu ibozlirki lxvuagx xon, er..."
26116,JHN 1:4,JHN,1,4,"In him was life, and the life was the light of...",In him was life; and the life was the light of...,In him was life; and the life was the light of...,"What came into existence in him was life, and ...","In hym was lijf, and the lijf was the liyt of ...","Hxel keni orlu ibozlirki or xon hez posi, erw ..."
26117,JHN 1:5,JHN,1,5,"The light shines in the darkness, and the dark...",And the light shineth in the darkness; and the...,And the light shineth in darkness; and the dar...,And the light goes on shining in the dark; it ...,and derknessis comprehendiden not it.,Erw lxi pogxl guiz ur zxororg or lxi wevc; ol ...
...,...,...,...,...,...,...,...,...,...,...
26987,JHN 21:21,JHN,21,21,"Peter, seeing him, said to Jesus, “Lord, what ...","Peter therefore seeing him saith to Jesus, Lor...","Peter seeing him saith to Jesus, Lord, and wha...","Seeing him, Peter said to Jesus, What about th...","Therfor whanne Petre hadde seyn this, he seith...","Ziiorg xon, Qiliv zeow lu Yizaz, Hxel efual lx..."
26988,JHN 21:22,JHN,21,22,"Jesus said to him, “If I desire that he stay u...","Jesus saith unto him, If I will that he tarry ...","Jesus saith unto him, If I will that he tarry ...","Jesus said to him, If it is my desire for him ...","Jhesus seith to him, So I wole that he dwelle ...","Yizaz zeow lu xon, Os ol oz nj wizovi suv xon ..."
26989,JHN 21:23,JHN,21,23,This saying therefore went out among the broth...,This saying therefore went forth among the bre...,Then went this saying abroad among the brethre...,So this saying went about among the brothers t...,Therfor this word wente out among the brithere...,Zu lxoz zejorg hirl efual enurg lxi fvulxivz l...
26990,JHN 21:24,JHN,21,24,This is the disciple who testifies about these...,This is the disciple that beareth witness of t...,This is the disciple which testifieth of these...,This is the disciple who gives witness about t...,"This is thilke disciple, that berith witnessyn...",Lxoz oz lxi wozkoqpi hxu gomiz holrizz efual l...


In [211]:
def rename_book(reference):
    book, remainder = reference.split()
    if book == 'MAT':
        book = 'Matthew'
    elif book == 'LUK':
        book = 'Luke'
    elif book == 'JHN':
        book = 'John'
    elif book == 'MRK':
        book = 'Mark'
    return " ".join([book, remainder])

PROMPT_SUFFIX = " from the Bible in the Berrig language says, \""
df2 = pd.DataFrame({'prompt': train['0'], 'completion': train['birrig']})
df2['completion'] = df2['completion'].apply(lambda x: " " + x + '"\n###')
df2['prompt'] = df2['prompt'].apply(lambda x: rename_book(x) + PROMPT_SUFFIX)
df2.to_json('data/train_exp1.jsonl', orient='records', lines=True)
df2


,prompt,completion
23213,Matthew 1:1 from the Bible in the Berrig langu...,"Lxi fuuc us lxi girivelourz us Yizaz Kxvozl, ..."
23214,Matthew 1:2 from the Bible in the Berrig langu...,Lxi zur us Efvexen hez Ozeek; erw lxi zur us ...
23215,Matthew 1:3 from the Bible in the Berrig langu...,Erw lxi zurz us Yawex hivi Qivid erw Divex fj...
23216,Matthew 1:4 from the Bible in the Berrig langu...,Erw lxi zur us Ven hez Ennorewef; erw lxi zur...
23217,Matthew 1:5 from the Bible in the Berrig langu...,Erw lxi zur us Zepnur fj Vexef hez Fued; erw ...
...,...,...
26987,John 21:21 from the Bible in the Berrig langua...,"Ziiorg xon, Qiliv zeow lu Yizaz, Hxel efual l..."
26988,John 21:22 from the Bible in the Berrig langua...,"Yizaz zeow lu xon, Os ol oz nj wizovi suv xon..."
26989,John 21:23 from the Bible in the Berrig langua...,Zu lxoz zejorg hirl efual enurg lxi fvulxivz ...
26990,John 21:24 from the Bible in the Berrig langua...,Lxoz oz lxi wozkoqpi hxu gomiz holrizz efual ...


In [201]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py tools fine_tunes.prepare_data -f data/train_exp1.jsonl -q

Analyzing...

- Your file contains 3101 prompt-completion pairs
- All prompts end with suffix ` from the Bible in the Berrig language says, "`. This suffix seems very long. Consider replacing with a shorter suffix, such as ` ->`
- All completions end with suffix `"\n###`

No remediations found.

You can use your file for fine-tuning:
> openai api fine_tunes.create -t "data/train_exp1.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string ` from the Bible in the Berrig language says, "` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=[""\n###"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 45.03 minutes to train a `curie` model, and less for `ada` and `babbage`. Queue will approximately take half an hour per job ahead of you.


In [207]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.create -t "data/train_exp1c.jsonl" -m "davinci"

Upload progress: 100%|███████████████████████| 660k/660k [00:00<00:00, 804Mit/s]
Uploaded file from data/train_exp1c.jsonl: file-3XqLJq8aIDSadDnW68fMi4TP
Created fine-tune: ft-pSvQW5yxPyHqU2eDrFDsaAGH
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-03-30 15:05:14] Created fine-tune: ft-pSvQW5yxPyHqU2eDrFDsaAGH

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-pSvQW5yxPyHqU2eDrFDsaAGH



In [209]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.follow -i ft-pSvQW5yxPyHqU2eDrFDsaAGH

[2023-03-30 15:05:14] Created fine-tune: ft-pSvQW5yxPyHqU2eDrFDsaAGH
[2023-03-30 15:08:49] Fine-tune costs $28.57
[2023-03-30 15:08:50] Fine-tune enqueued. Queue number: 29
[2023-03-30 15:09:34] Fine-tune is in the queue. Queue number: 28
[2023-03-30 15:10:37] Fine-tune is in the queue. Queue number: 27
[2023-03-30 15:10:46] Fine-tune is in the queue. Queue number: 26
[2023-03-30 15:11:42] Fine-tune is in the queue. Queue number: 25
[2023-03-30 15:13:15] Fine-tune is in the queue. Queue number: 24
[2023-03-30 15:13:48] Fine-tune is in the queue. Queue number: 23
[2023-03-30 15:14:04] Fine-tune is in the queue. Queue number: 22
[2023-03-30 15:14:25] Fine-tune is in the queue. Queue number: 21
[2023-03-30 15:16:19] Fine-tune is in the queue. Queue number: 20
[2023-03-30 15:16:47] Fine-tune is in the queue. Queue number: 19
[2023-03-30 15:17:22] Fine-tune is in the queue. Queue number: 18
[2023-03-30 15:18:34] Fine-tune is in the queue. Queue number: 17
[2023-03-30 15:19:04] Fine-tune is 

In [220]:
import openai
from lib.cipher import substitution_cipher
from nltk.translate.bleu_score import sentence_bleu

openai.api_key = os.getenv("OPENAI_API_KEY")

def question(text, model='text-davinci-003', stop=["\n###"]):
    response = openai.Completion.create(
        model=model,
        prompt=text,
        temperature=0.15,  # 0.1 is very little randomness/creativity, 2 is very
        max_tokens=300,
        frequency_penalty=0,
        presence_penalty=0,
        logprobs=3,
        stop=stop
    )
    
    
    return response.get('choices',[])[0].get('text','').strip()

def translate(text):
    print(text + PROMPT_SUFFIX)
    response = openai.Completion.create(
        model="davinci:ft-personal-2023-03-30-23-29-22",
        prompt=text + PROMPT_SUFFIX,
        temperature=0.5,  # 0.1 is very little randomness/creativity, 2 is very
        max_tokens=100,
        n=1,
        frequency_penalty=0,
        presence_penalty=0,
        #stop=["\n###"]
    )
    result = []
    for choice in response.get('choices',[{}]):
        decoded = substitution_cipher(choice.get('text').strip(), encode=False).strip()
        result.append({
            'translation': choice.get('text').strip(),
            'decoded_translation': decoded,
            #'bleu_score': sentence_bleu([text.split()], decoded.split())
        })
    
    return result

In [ ]:
question("In the beginnning was the ")

In [217]:
# Fails to do so
translate("John 3:16")

John 3:16 from the Bible in the Berrig language says, "


[{'translation': 'Lxir lxi Yihz zeow lu xon, Lxi Yihz xez ruh wuri. Erw lxij zeow lu xon, Wu jua ruh wuri?"\n### Erw lxij qal xon lu wielx, erw lxij gemi xon fpuhz lu lxin.\n### Lxij zeow lu xon, Hxel oz x',
  'decoded_translation': 'Then the Jews said to him, The Jews has now done. And they said to him, Do you now done?"\n### And they put him to death, and they gave him blows to them.\n### They said to him, What is h'}]

In [222]:
question("Quote John 3:16 in several English Bibles \"", "davinci:ft-personal-2023-03-30-23-29-22")

'For God so loved the world that he gave his only begotten Son, that whoever believes in him should not perish, but have eternal life."'

## Here we test if GPT has knowledge of a verse on it's own
Learnings
 - John works much better than JHN
 - Davini is not as good as text-davini-003 ???
 - if you rewrite it for a 5 year old it shortens it
 - without a for a 9 year old it chooses NIV or KJV or other
 - confuses John 18:2 for John 18:4
 - it can pull the gender, plurarily and others for the text (to be confirmed if correct though)

In [197]:
question("John 18:2 from the Bible in the French language says, \"")

'Toutes les cohortes se sont mises en mouvement, et ils ont pris Jésus." This translates to, "All the cohorts set out, and they took Jesus."'

In [187]:
question("John 18:2 from the Bible rewritten for 9 year olds without losing any keywords says, \"")

'And Jesus, knowing all that was going to happen to Him, went out to meet His betrayer."'

In [188]:
question("John 18:2 from the Bible in 3 languages, \"")

'English: And Judas also, which betrayed him, knew the place: for Jesus ofttimes resorted thither with his disciples.\n\nSpanish: Y Judas, el que le entregó, conocía también aquel lugar, porque Jesús se había reunido allí muchas veces con sus discípulos.\n\nGerman: Und auch Judas, der ihn verriet, kannte den Ort; denn Jesus war oft dorthin mit seinen Jüngern gekommen.'

In [194]:
question("1 John 1:1 in English but mark plural as (plural) and gender from greek as (Masc, Fem, or Neuter) for every word\"")

'That which was from the (Neuter) beginning, which we have heard, which we have seen with our (plural) eyes, which we have looked upon, and our (plural) hands have handled, of the Word of life;'

In [190]:
question("John 18:2 in English but mark plural as (plural) and gender from greek as (Masc, Fem, or Neuter), \"")

'And the (Masc) servants and officers (plural) came to him, and the (Masc) Judas also, which betrayed him."'

## Feedback
In theory we thought that it could map just from the reference b/c it shows that on devinci it knows the verse.  So in theory it should have all the versions of that text, all the sermons and all the commentaries and comments about it embedded into it.  But when asked for that verse after training it then it doesn't know it so it appears to not be packaging all those findings with it.

We did a second attempt where we set the prompt to John 3:16 in the Bible says, " but even at that it was confused and we could not extract the internal knowledge in fine tuning only in directly calling it.